# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [14]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time
import gmaps.datasets

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
weather = pd.read_csv("../outputs/weather.csv",index_col=0)
weather.head()

,city,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,tilichiki,60.47,166.10,50.38,96,100,4.34,RU,1596985591
1,punta arenas,-53.15,-70.92,37.40,64,100,17.22,CL,1596985424
2,half moon bay,37.46,-122.43,62.60,100,90,5.82,US,1596985593
3,bengkulu,-3.80,102.27,78.30,86,100,6.67,ID,1596985594
4,ribeira grande,38.52,-28.70,77.00,65,20,11.41,PT,1596985595


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
gmaps.configure(api_key=g_key)

In [6]:
import gmaps
gmaps.configure(api_key=g_key)
locations = weather[['Lat', 'Lng']].astype(float)
weights = weather['Humidity']
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
weather =weather.dropna()

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = weather[(weather['Max Temp']>70) &(weather['Max Temp']<85)]
hotel_df= hotel_df.dropna()
hotel_df["Hotel Name"]= ""
hotel_df.head(5)

,city,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
3,bengkulu,-3.80,102.27,78.30,86,100,6.67,ID,1596985594,
4,ribeira grande,38.52,-28.70,77.00,65,20,11.41,PT,1596985595,
5,san juan de los morros,9.91,-67.35,79.18,82,39,3.20,VE,1596985596,
6,kapaa,22.08,-159.32,78.80,83,75,16.11,US,1596985598,
7,kavieng,-2.57,150.80,81.36,76,100,4.43,PG,1596985599,


In [9]:
test = hotel_df['Lat']

In [16]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
hotels =[]
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]


    params = {
    "location": f"{lat},{lng}",
    "radius": 5000,
    "types" : "hotel",
    "keyword": "hotel",
    "key": g_key}
    #print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(url, params=params).json()
    
    results = response['results']
    
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    except (KeyError, IndexError):
        print("No hotels within specified radius... skipping.")
        
     #time.sleep(1)


print("-------End of Search-------")

No hotels within specified radius... skipping.
No hotels within specified radius... skipping.
No hotels within specified radius... skipping.
No hotels within specified radius... skipping.
No hotels within specified radius... skipping.
No hotels within specified radius... skipping.
No hotels within specified radius... skipping.
No hotels within specified radius... skipping.
No hotels within specified radius... skipping.
No hotels within specified radius... skipping.
No hotels within specified radius... skipping.
No hotels within specified radius... skipping.
No hotels within specified radius... skipping.
No hotels within specified radius... skipping.
No hotels within specified radius... skipping.
No hotels within specified radius... skipping.
No hotels within specified radius... skipping.
No hotels within specified radius... skipping.
No hotels within specified radius... skipping.
No hotels within specified radius... skipping.
No hotels within specified radius... skipping.
No hotels wit

In [10]:
hotel_df

,city,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
3,bengkulu,-3.80,102.27,78.30,86,100,6.67,ID,1596985594,
4,ribeira grande,38.52,-28.70,77.00,65,20,11.41,PT,1596985595,
5,san juan de los morros,9.91,-67.35,79.18,82,39,3.20,VE,1596985596,
6,kapaa,22.08,-159.32,78.80,83,75,16.11,US,1596985598,
7,kavieng,-2.57,150.80,81.36,76,100,4.43,PG,1596985599,
...,...,...,...,...,...,...,...,...,...,...
589,hambantota,6.12,81.12,77.00,94,90,11.41,LK,1596986283,
591,dunmore town,25.50,-76.65,83.79,71,27,6.42,BS,1596986285,
594,touros,-5.20,-35.46,80.60,69,75,12.75,BR,1596986288,
595,katobu,-4.94,122.53,75.24,92,86,4.97,ID,1596986289,


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(layer)


fig



Figure(layout=FigureLayout(height='420px'))